In [1]:
import datetime
import folium
import importlib
import pandas as pd
import statistics
import tqdm

import SaveEcoBotFinder
import IdMatcher
import CamsWrapper

importlib.reload(SaveEcoBotFinder)
importlib.reload(IdMatcher)
importlib.reload(CamsWrapper)

from SaveEcoBotFinder import SaveEcoBotFinder
from IdMatcher import IdMatcher
from CamsWrapper import CamsWrapper

MATCHER = IdMatcher("співставлення ID постів з ID Cams.csv", bannedStations=[])
FINDER = SaveEcoBotFinder("/media/osamoile/data/Ecobot_16_11_2020")
# CAMS = CamsWrapper("Cams_2019_2020_pm2.5_Kyiv.csv")

In [2]:
fullDf = pd.read_csv("/media/osamoile/data/all_posts.csv")
fullDf.sort_values(by=["date", "hour", "minute", "second"], inplace=True)
fullDf

,device_id,phenomenon,value,logged_at,value_text,date,hour,minute,second
2057952,28,pm25,14.2200,2019-03-10 00:03:38,NaN,2019-03-10,0,3,38
2482767,30,pm25,24.0200,2019-03-10 00:03:46,NaN,2019-03-10,0,3,46
2057953,28,pm25,14.9200,2019-03-10 00:06:09,NaN,2019-03-10,0,6,9
2482768,30,pm25,22.6000,2019-03-10 00:06:14,NaN,2019-03-10,0,6,14
2057954,28,pm25,15.2000,2019-03-10 00:08:37,NaN,2019-03-10,0,8,37
...,...,...,...,...,...,...,...,...,...
8228593,13141,pm25,29.9500,2020-11-16 22:13:39,NaN,2020-11-16,22,13,39
8470855,13802,pm25,40.3462,2020-11-16 22:13:39,NaN,2020-11-16,22,13,39
11191987,13207,pm25,28.1667,2020-11-16 22:13:39,NaN,2020-11-16,22,13,39
15455361,13208,pm25,29.6296,2020-11-16 22:13:39,NaN,2020-11-16,22,13,39


1. Data about pm25 pollutant
2. Data from 2019-03-10 till 2020-11-16 - totally 20 months

In [13]:
unique_list = fullDf.device_id.unique()

cams_ids = []
for i in unique_list:
    cams_ids.append(MATCHER.getCamsIdByPostId(i))

postCamsMatch = pd.DataFrame({"device_id": unique_list, "cams_id": cams_ids})
postCamsMatch

,device_id,cams_id
0,28,6563
1,30,6562
2,43,6564
3,47,6378
4,108,6747
...,...,...
146,13802,6378
147,13803,6564
148,13811,6564
149,13820,6379


In [20]:
unique_cams_list = postCamsMatch.cams_id.unique()
len(unique_cams_list)

12

In [19]:
postCamsMatch.groupby(postCamsMatch.cams_id).count().sort_values("device_id", ascending=False)

,device_id
cams_id,
6378,39
6564,32
6563,31
6377,12
6379,11
6749,9
6193,5
6194,4
6562,4


Data from 151 post in 12 pixels

!!! Show schema in slides

In [27]:
res = fullDf.groupby(fullDf.device_id).date.apply(lambda x: len(x.unique())).sort_values(ascending=False)
res
#res.head(10)

device_id
30      617
43      610
28      491
914     482
915     467
       ... 
1415      2
3600      2
1330      1
3709      1
3111      1
Name: date, Length: 151, dtype: int64

In [28]:
res.describe()

count    151.000000
mean     177.927152
std      135.395869
min        1.000000
25%       62.500000
50%      170.000000
75%      264.500000
max      617.000000
Name: date, dtype: float64

Amount of days when post is active is from 1 to 617 days with median 170 days

In [32]:
res = fullDf.groupby([fullDf.device_id, fullDf.date]).value.count()
res.sort_values(ascending=False)

device_id  date      
108        2019-04-21    1257
43         2019-04-22    1252
108        2019-04-20    1252
43         2019-04-19    1252
108        2019-04-19    1244
                         ... 
3690       2020-08-07       1
3687       2020-05-13       1
3690       2020-08-22       1
3692       2020-05-13       1
1387       2020-03-17       1
Name: value, Length: 26867, dtype: int64

In [33]:
res.describe()

count    26867.000000
mean       606.522909
std        320.866544
min          1.000000
25%        432.000000
50%        557.000000
75%        949.000000
max       1257.000000
Name: value, dtype: float64

Amount of station logs in from 1 to 1257 per hour (~ 1 log per 1.5 minutes)

Median is 557 logs per day (~ 1 log per 3 minutes)

In [64]:
post30 = pd.read_csv("/media/osamoile/data/Ecobot_16_11_2020/saveecobot_30.csv")
post30.phenomenon.unique()

array(['pm10', 'pm25', 'humidity', 'temperature'], dtype=object)